In [318]:
import tensorflow as tf
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [319]:
%%bash
ls

__init__.py
foil2.ipynb


In [327]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [372]:
num_entities = 2
num_relations = 3
emb_dim = 4
ent_embeddings = tf.Variable(tf.random_normal([num_entities, emb_dim],stddev=0.1), tf.float32)
rel_embeddings = tf.Variable(tf.random_normal([num_relations, emb_dim, emb_dim],stddev=0.1), tf.float32)

bool_ent_embeddings = tf.sigmoid(ent_embeddings)
parameters = [ent_embeddings,rel_embeddings]

def scores(rel_ids, arg1_ids, arg2_ids):
    batch_rel_embeddings = tf.gather(rel_embeddings, rel_ids) # [num_facts, emb_dim, emb_dim]
    arg1_embeddings = tf.gather(bool_ent_embeddings, arg1_ids)
    arg2_embeddings = tf.gather(bool_ent_embeddings, arg2_ids) # [num_facts, 1, emb_dim]
    return scores_for_embeddings(batch_rel_embeddings, arg1_embeddings, arg2_embeddings)

def scores_for_embeddings(batch_rel_embeddings, arg1_embeddings,arg2_embeddings):
    expanded_arg1_embeddings = tf.expand_dims(arg1_embeddings, 2)
    expanded_arg2_embeddings = tf.expand_dims(arg2_embeddings, 1)
    result = tf.reduce_sum(expanded_arg1_embeddings * batch_rel_embeddings * expanded_arg2_embeddings, [1,2])
    return result

def log_loss(scores, targets):
    return tf.nn.sigmoid_cross_entropy_with_logits(scores, targets)

num_order_2_rules = 2
use_linear = True
if use_linear:
    rule_violators_arg1_raw = tf.Variable(tf.random_uniform([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg2_raw = tf.Variable(tf.random_uniform([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg1 = tf.maximum(0., tf.minimum(1., rule_violators_arg1_raw))
    rule_violators_arg2 = tf.maximum(0., tf.minimum(1., rule_violators_arg2_raw))
else:
    rule_violators_arg1_raw = tf.Variable(tf.random_normal([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg2_raw = tf.Variable(tf.random_normal([num_order_2_rules, emb_dim]), tf.float32) 
    rule_violators_arg1 = tf.sigmoid(rule_violators_arg1_raw)
    rule_violators_arg2 = tf.sigmoid(rule_violators_arg2_raw)

rule_violators = [rule_violators_arg1_raw, rule_violators_arg2_raw]

def rule_losses(heads, bodies):
    head_embeddings = tf.gather(rel_embeddings, heads) # [num_rules, emb_dim, emb_dim]
    body_embeddings = tf.gather(rel_embeddings, bodies) # [num_rules, emb_dim, emb_dim]
    rule_head_scores = scores_for_embeddings(head_embeddings,rule_violators_arg1, rule_violators_arg2)
    rule_body_scores = scores_for_embeddings(body_embeddings,rule_violators_arg1, rule_violators_arg2)
    losses = tf.maximum(0.0, rule_body_scores - rule_head_scores)
    return losses

In [322]:
sess.run(tf.initialize_all_variables())
# sess.run(log_loss(scores([0,1],[1,2],[2,3]), [1.,0]))

In [323]:
sess.run(rule_losses([0,1],[1,2]))

array([ 0.01249681,  0.        ], dtype=float32)

In [384]:
opt_learn = tf.train.AdamOptimizer(learning_rate=0.01)
opt_find = tf.train.AdamOptimizer(learning_rate=1.0)

rule_violation_loss = rule_losses([1,2],[0,1])
training_loss = log_loss(scores([0],[0],[1]),[1.]) + 100 * rule_violation_loss + \
    0.001 * tf.nn.l2_loss(rel_embeddings) + \
    0.001 * tf.nn.l2_loss(ent_embeddings)

opt_find_violation = opt_find.minimize(-rule_violation_loss,var_list=rule_violators)
opt_find_parameters = opt_learn.minimize(training_loss, var_list=parameters)

In [401]:
sess.run(tf.initialize_all_variables())
num_outer_epochs = 10
num_find_violations_epochs = 10
num_training_epochs = 10
num_print_out = 1
for outer_epoch in range(0,num_outer_epochs):
    print("Estimate parameters")
    for i in range(0,num_training_epochs):
        current_loss,_ = sess.run([training_loss,opt_find_parameters])
#         if i % (num_training_epochs // num_print_out) == 0:
#             print(current_loss)

    print(sess.run(training_loss))
    print(sess.run(scores([0,1,2],[0,0,0],[1,1,1])))

    print("Find Violations")
    sess.run(tf.initialize_variables(rule_violators))
    for i in range(0,num_find_violations_epochs):
        current_loss,_ = sess.run([rule_violation_loss,opt_find_violation])
#         if i % (num_find_violations_epochs // num_print_out) == 0:
#             print(current_loss)
    print(sess.run(rule_violation_loss))
                
sess.run(scores([0,1,2],[0,0,0],[1,1,1])) 
# sess.run(scores([1],[0],[1])) 

Estimate parameters
[ 0.72563934  0.72563934]
[-0.06332336 -0.1491591   0.31144169]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.73712397  0.73712397]
[-0.08528074 -0.19883703  0.45219451]
Find Violations
[ 0.75440991  0.        ]
Estimate parameters
[ 0.83022487  0.83022487]
[-0.25669122 -0.02310122  0.47937843]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.85765785  0.85765785]
[-0.30455074  0.05977622  0.44744927]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.84552252  0.84552252]
[-0.28293148  0.07878168  0.39898714]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.8240664  0.8240664]
[-0.24439813  0.07693154  0.35198343]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.80353218  0.80353218]
[-0.20676219  0.06974072  0.31207442]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.7863273  0.7863273]
[-0.1745529   0.06214888  0.27999753]
Find Violations
[ 0.  0.]
Estimate parameters
[ 0.77238917  0.77238917]
[-0.14793487  0.05547256  0.25482884]
Find Violations
[ 0.  0.

array([-0.12594725,  0.04989794,  0.23531279], dtype=float32)